In [ ]:
import pathlib
import os
import requests
from urllib.parse import urlparse, parse_qs
from io import BytesIO
import pandas as pd
import json

# city api https://public.opendatasoft.com/

In [ ]:
GET_REQUEST = "https://www.eco-visio.net/api/aladdin/1.0.0/pbl/publicwebpageplus/4586?withNull=true"

In [3]:
url_endpoint = GET_REQUEST
response = requests.get(url_endpoint)
response

<Response [200]>

In [4]:
data_json = json.loads(response.content)

In [5]:
data_json[0]

{'token': None,
 'id_pdc_img': 0,
 'idPdc': 100021443,
 'lat': 69.96787,
 'lon': 23.36929,
 'nom': 'Alta_2 (NO)',
 'publicMessage': None,
 'photo': [{'lien': 'https://filer.eco-counter-tools.com/file/c3/a6b6e6ded8b909892faafb44eebf6883120ef9ab7695ccb8aca0401899b1fec3/14315298005820.jpg'}],
 'lienPublic': None,
 'pratique': [{'pratique': 2, 'id': 101021443},
  {'pratique': 2, 'id': 102021443}],
 'mainPratique': 2,
 'filtre': None,
 'formule_site': None,
 'fin': None,
 'debut': '01/01/2013',
 'finPeriode': None,
 'debutPeriode': '01/01/2021',
 'current_year_default': 1,
 'externalUrl': 'https://www.eco-compteur.com',
 'nomOrganisme': 'Bike Count Display Interactive Map',
 'logo': 'https://filer.eco-counter-tools.com/file/33/43649b3090eaa41e49d73bb31ff037efe2f97745f8a8e8eafb3baab42acf6433/1556292443839.png',
 'pays': 'fr',
 'sig': 1,
 'pictoPublic': 'totem_vert.png',
 'today': '27/09/2021',
 'total': 33606,
 'lastDay': 79,
 'moyD': 125,
 'totalLY': None,
 'lastDay8': None,
 'moyDLY': None

In [6]:
with open('counting_stations_metadata_global.json', 'w') as outfile:
    json.dump(data_json, outfile)

In [7]:
file = open('counting_stations_metadata_global.json',)
counting_stations_metadata_global = json.load(file)

In [54]:
counting_stations_metadata_global[0]

{'token': None,
 'id_pdc_img': 0,
 'idPdc': 100021443,
 'lat': 69.96787,
 'lon': 23.36929,
 'nom': 'Alta_2 (NO)',
 'publicMessage': None,
 'photo': [{'lien': 'https://filer.eco-counter-tools.com/file/c3/a6b6e6ded8b909892faafb44eebf6883120ef9ab7695ccb8aca0401899b1fec3/14315298005820.jpg'}],
 'lienPublic': None,
 'pratique': [{'pratique': 2, 'id': 101021443},
  {'pratique': 2, 'id': 102021443}],
 'mainPratique': 2,
 'filtre': None,
 'formule_site': None,
 'fin': None,
 'debut': '01/01/2013',
 'finPeriode': None,
 'debutPeriode': '01/01/2021',
 'current_year_default': 1,
 'externalUrl': 'https://www.eco-compteur.com',
 'nomOrganisme': 'Bike Count Display Interactive Map',
 'logo': 'https://filer.eco-counter-tools.com/file/33/43649b3090eaa41e49d73bb31ff037efe2f97745f8a8e8eafb3baab42acf6433/1556292443839.png',
 'pays': 'fr',
 'sig': 1,
 'pictoPublic': 'totem_vert.png',
 'today': '26/09/2021',
 'total': 33527,
 'lastDay': 43,
 'moyD': 125,
 'totalLY': None,
 'lastDay8': None,
 'moyDLY': None

In [47]:
def get_position_of_brackets(String):
    first = String.find('(')
    second = String.find(')')
    return [first,second]

def get_country_between_brackets(String, brackets_index):
    country = String[brackets_index[0]+1: brackets_index[1]]
    return country

def get_city_and_number_from_name(String):
    first, second = get_position_of_brackets(String)
    city = String[:first].strip()
    index = String.find('_')
    number = 1
    if index != -1:
        number = city[index+1]
        city = city[:index]
    return city, number
    
print(get_city_and_number_from_name("Heidelberg_2 (DE)"))

('Heidelberg', '2')


In [48]:
name_list = []
number_list=[]
latitude_list = []
longitude_list= []
id_list = []
columns = ["name", "number", "latitude", "longitude", "id_list"]

In [49]:
german_counting_stations = []
for counting_station in counting_stations_metadata_global:
    name = counting_station["nom"]
    brackets_index = get_position_of_brackets(name)
    country = get_country_between_brackets(name, brackets_index)
    if country == "DE":
        german_counting_stations.append(counting_station)
        id_list.append(counting_station["id_pdc_img"])
        station_name ,station_number = get_city_and_number_from_name(counting_station["nom"])
        name_list.append(station_name)
        number_list.append(station_number)
        latitude_list.append(counting_station["lat"])
        longitude_list.append(counting_station["lon"])

In [51]:
df = pd.DataFrame(
    list(zip(name_list, number_list, 
             latitude_list, longitude_list, 
             id_list)),
               columns =columns)

In [52]:
df

,name,number,latitude,longitude,id_list
0,Sommerda,1,51.287374,11.060599,100055269
1,Stuttgart,1,48.784240,9.147031,100063203
2,Stuttgart,2,48.826000,9.214880,100063205
3,Stuttgart,3,48.716494,9.086520,100061257
4,Stuttgart,4,48.739821,9.152228,100061633
5,Stuttgart,5,48.740003,9.226692,100061648
6,Stuttgart,6,48.811509,9.167497,100062943
7,Stuttgart,7,48.809354,9.105381,100062945
8,Stuttgart,8,48.779637,9.248607,100063204
9,Arnsberg,1,51.440702,7.964296,100063831


In [53]:
df.to_csv("counting_stations_germany_metadata.csv")